In [17]:
import openai
import os
import pandas as pd
import requests
import tinify
from ast import literal_eval
import sqlalchemy
import psycopg2

rijks_api_key = "enter_rijks_api_key"

In [2]:
collection = pd.read_csv('collection.csv')

In [3]:
collection.head()

,objectInventoryNumber,objectPersistentIdentifier,objectTitle[1],objectType[1],objectCreator[1],objectCreationDate[1],objectImage
0,-1RP-F-2001-7-1480,http://hdl.handle.net/10934/RM0001.COLLECT.704235,Les artistes belges,boek,NaN,NaN,NaN
1,2RP-F-2001-7-1020-41,http://hdl.handle.net/10934/RM0001.COLLECT.678849,Portret van Prescott H. Butler,fotomechanische afdruk,anonymous,1869,NaN
2,AK-BR-324,http://hdl.handle.net/10934/RM0001.COLLECT.321537,"Klokvormige kom met bleu poudré, draken en parels",kom,anonymous,1800,https://lh3.googleusercontent.com/S3wgRasrmmAk...
3,AK-BR-325,http://hdl.handle.net/10934/RM0001.COLLECT.3866,Eivormige vaas met een rood glazuur,vaas,anonymous,1800,https://lh3.googleusercontent.com/kc2u8EkKqRbD...
4,AK-BR-367,http://hdl.handle.net/10934/RM0001.collect.163127,Ronde doos met goudlak,doos,NaN,1775,https://lh3.ggpht.com/JNbJGaB7EIJqolKXACtFVcYZ...


In [4]:
len(collection)

667894

In [5]:
collection.dropna(subset=['objectInventoryNumber', 'objectImage'], how='any', inplace=True)

In [6]:
len(collection)

392639

In [7]:
collection.iloc[90,0]

'AK-KOG-315-B'

In [8]:
rembrandt_collection = collection[collection['objectCreator[1]'].str.contains('Rembrandt', case=False, na=False)]
rembrandt_collection

,objectInventoryNumber,objectPersistentIdentifier,objectTitle[1],objectType[1],objectCreator[1],objectCreationDate[1],objectImage
13549,BI-1961-168-1,http://hdl.handle.net/10934/RM0001.COLLECT.37116,Gebaarde oude man met hoge bontmuts en geslote...,prent,"Rijn, Rembrandt van",1632,https://lh3.googleusercontent.com/r-5IkD0BotEK...
13550,BI-1961-168-10,http://hdl.handle.net/10934/RM0001.COLLECT.37125,Jacob liefkoost Benjamin,prent,"Rijn, Rembrandt van",1635,https://lh3.googleusercontent.com/SH2yc6hml1g7...
13552,BI-1961-168-101,http://hdl.handle.net/10934/RM0001.COLLECT.37216,Portret van de Remonstrantse predikant Jan Wte...,prent,"Rijn, Rembrandt van",1635,https://lh3.googleusercontent.com/u84liMHYCKl8...
13554,BI-1961-168-103,http://hdl.handle.net/10934/RM0001.COLLECT.37218,Doop van de kamerling,prent,"Rijn, Rembrandt van",1641,https://lh3.googleusercontent.com/BBxhu76NBJNB...
13556,BI-1961-168-105,http://hdl.handle.net/10934/RM0001.COLLECT.37220,Portret van de apotheker Abraham Francen,prent,"Rijn, Rembrandt van",1655,https://lh3.googleusercontent.com/27NVp8Fy-rUu...
...,...,...,...,...,...,...,...
666015,SK-C-1768,http://hdl.handle.net/10934/RM0001.COLLECT.616582,Portret van Oopjen Coppit,schilderij,"Rijn, Rembrandt van",1634,https://lh3.googleusercontent.com/ib8SNTK2Qk-z...
666070,SK-C-216,http://hdl.handle.net/10934/RM0001.COLLECT.5223,"Portret van een paar als Isaak en Rebekka, bek...",schilderij,"Rijn, Rembrandt van",1665,https://lh3.googleusercontent.com/mAyAjvYjIeAI...
666193,SK-C-5,http://hdl.handle.net/10934/RM0001.COLLECT.5216,Officieren en andere schutters van wijk II in ...,schilderij,"Rijn, Rembrandt van",1642,https://lh3.googleusercontent.com/J-mxAE7CPu-D...
666240,SK-C-597,http://hdl.handle.net/10934/RM0001.COLLECT.5237,"Portret van een vrouw, mogelijk Maria Trip",schilderij,"Rijn, Rembrandt van",1639,https://lh3.googleusercontent.com/AyiKhdEWJ7Xm...


In [9]:
collection['objectType[1]'].unique()

array(['kom', 'vaas', 'doos', ..., 'modetekening', 'heraldisch object',
       'schilderijlijst'], dtype=object)

In [10]:
collection['objectType[1]'].value_counts()

objectType[1]
prent                     194299
foto                       47159
tekening                   30008
schetsboekblad             26650
fotomechanische afdruk     14497
                           ...  
onderkast                      1
kinderstoel                    1
rekenbord                      1
bruidshart                     1
voile                          1
Name: count, Length: 2036, dtype: int64

In [11]:
# just paintings 'schilderij'

rembrandt_collection_paintings = rembrandt_collection[rembrandt_collection['objectType[1]'] == 'schilderij']

In [12]:
rembrandt_collection_paintings

,objectInventoryNumber,objectPersistentIdentifier,objectTitle[1],objectType[1],objectCreator[1],objectCreationDate[1],objectImage
661912,SK-A-1935,http://hdl.handle.net/10934/RM0001.COLLECT.5219,Landschap met stenen brug,schilderij,"Rijn, Rembrandt van",1638,https://lh3.googleusercontent.com/_s3YWiBaZ5ML...
662400,SK-A-2391,http://hdl.handle.net/10934/RM0001.COLLECT.5235,Jongenskopje,schilderij,"Rijn, Rembrandt van",1643,https://lh3.googleusercontent.com/bKmQJHy1JbMH...
663119,SK-A-3066,http://hdl.handle.net/10934/RM0001.COLLECT.5243,"Oude lezende vrouw, waarschijnlijk de profetes...",schilderij,"Rijn, Rembrandt van",1631,https://lh3.googleusercontent.com/-puHtaElxzr8...
663197,SK-A-3137,http://hdl.handle.net/10934/RM0001.COLLECT.5222,De verloochening van Petrus,schilderij,"Rijn, Rembrandt van",1660,https://lh3.googleusercontent.com/OUqLlJidxoD9...
663198,SK-A-3138,http://hdl.handle.net/10934/RM0001.COLLECT.5227,Titus in monniksdracht,schilderij,"Rijn, Rembrandt van",1660,https://lh3.googleusercontent.com/N9vZO35XvXWc...
663306,SK-A-3276,http://hdl.handle.net/10934/RM0001.COLLECT.5242,Jeremia treurend over de verwoesting van Jeruz...,schilderij,"Rijn, Rembrandt van",1630,https://lh3.googleusercontent.com/lLBt3A_Kvwwy...
663326,SK-A-3298,http://hdl.handle.net/10934/RM0001.COLLECT.5229,"Portret van een man, misschien Rembrandts vade...",schilderij,"Rijn, Rembrandt van",1730,https://lh3.googleusercontent.com/2RbREkrUQstt...
663374,SK-A-3340,http://hdl.handle.net/10934/RM0001.COLLECT.5218,Borstbeeld van een man in oosterse kleding,schilderij,"Rijn, Rembrandt van",1635,https://lh3.googleusercontent.com/4grKP_pusgY7...
663375,SK-A-3341,http://hdl.handle.net/10934/RM0001.COLLECT.5228,Portret van Rembrandt Harmensz. van Rijn,schilderij,"Rijn, Rembrandt van",1640,https://lh3.googleusercontent.com/nTZOJdH_heYs...
663518,SK-A-3477,http://hdl.handle.net/10934/RM0001.collect.5238,Jozef zijn dromen vertellend,schilderij,"Rijn, Rembrandt van",1633,https://lh3.googleusercontent.com/kbErvwypmDua...


In [110]:
from IPython.display import display, HTML

def format_quiz_response(quiz_text):
    questions = quiz_text.split('\n\n')
    
    formatted_questions = []
    for question in questions:
        question = question.replace('Correct Answer:', '<b>Correct Answer:</b>')
        formatted_question = '<p>' + question.replace("\n", "<br>") + '</p>'
        formatted_questions.append(formatted_question)
    
    return ''.join(formatted_questions)

In [112]:
# With only the painting

client = openai.OpenAI(
    api_key = 'enter_api_key'
)

response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "For this painting, give two multiple choice questions with 4 possible answers and label the right answer? These questions should be possibly answered by a human who takes a look at the image"},
        {
          "type": "image_url",
          "image_url": {
            "url": rembrandt_collection_paintings.iloc[10, 6],
          },
        },
      ],
    }
  ],
  max_tokens=1200,
)

quiz_response = response.choices[0].message.content
formatted_response = format_quiz_response(quiz_response)
display(HTML(formatted_response))


In [113]:
rembrandt_collection_paintings.iloc[10, 6]

'https://lh3.googleusercontent.com/nBnAOkixdJMLJnUUzZzqiFrqnJTAEuvWnrjwtQ3FUHUz1dSeMK3zseWvuFgWJldLDJFFEDtPprBr3zLkfqOK9MdT6A=s0'

In [124]:
def fetch_object_details(object_name, api_key="?key=enter_rijks_api_key", endpoint="https://www.rijksmuseum.nl/api/en/collection/"):
    
    url = endpoint + object_name + api_key
    headers = {"accept": "*/*", "accept-language": "en-US,en;q=0.9", "User-Agent": "PostmanRuntime/7.36.1", "Connection" : "keep-alive"}

    try:
        response = requests.request("GET", url, headers=headers)
        response.raise_for_status()
        data = response.json()
        #print(data)
        if data['artObject']:
            first_object = data['artObject']
            #print(data)
            return {
                'title': first_object.get('title', 'No title available'),
                'image_url': first_object.get('webImage', {}).get('url', 'No image URL available'),
                'description': first_object.get('plaqueDescriptionEnglish', 'No description available'),
                'add_description': first_object.get('label', {}).get('description', 'No additional description available')
            }
        else:
            return {'title': 'Not found', 'image_url': '', 'description': ''}

    except requests.RequestException as e:
        return {'error': f"API request failed: {e}"}


In [125]:
print(fetch_object_details('SK-C-6'))

{'title': 'The Sampling Officials of the Amsterdam Drapers’ Guild, Known as ‘The Syndics’', 'image_url': 'https://lh3.googleusercontent.com/gShVRyvLLbwVB8jeIPghCXgr96wxTHaM4zqfmxIWRsUpMhMn38PwuUU13o1mXQzLMt5HFqX761u8Tgo4L_JG1XLATvw=s0', 'description': 'Samplers checked the quality of dyed cloth. Here Rembrandt shows them at work, distracted for a moment and looking up. One syndic is about to sit, or stand, so not all the heads are at the same level. A clever trick which, with the confident brushwork and subtle use of light, make this one of the liveliest group portraits of the 17th century. Originally painted for the sampling hall (Staalhof), in 1771 it was acquired by Amsterdam’s town hall.', 'add_description': 'After suffering financial difficulties in the 1650s, Rembrandt moved to a rental house on the Rozengracht. The Amsterdam élite no longer knocked on his door as often as they had done before. He nevertheless remained popular: this important guild commissioned him to paint a gro

In [126]:
painting = fetch_object_details(rembrandt_collection_paintings.iloc[27,0])
print(painting)
painting['description']
full_text = 'Title of the painting is ' + painting['title'] + ' and the description of it is ' + painting['description']
full_text_add = 'Title of the painting is ' + painting['title'] + ' and the description of it is ' + painting['description'] + ' More description is ' + painting['add_description']

{'title': 'Isaac and Rebecca, Known as ‘The Jewish Bride’', 'image_url': 'https://lh3.googleusercontent.com/mAyAjvYjIeAIlByhJx1Huctgeb58y7519XYP38oL1FXarhVlcXW7kxuwayOCFdnwtOp6B6F0HJmmws-Ceo5b_pNSSQs=s0', 'description': 'It seems that Rembrandt painted his subjects as the biblical couple, Isaac and Rebecca. Its popular name, the Jewish Bride, is a later invention.\r\nThe portrait is painted with an extraordinarily free hand, as in the sleeve, where the paint is especially thick and shaped to reflect the light.', 'add_description': 'To prevent being killed and having his wife captured by King Abimelech, Isaac concealed his love for Rebecca by pretending they were brother and sister. However, their intimacy betrayed them when they thought they were not being spied on. Rembrandt depicts them in a tender moment. Furthermore, he works with exceptional freedom, applies the paint thickly, and scratches into it with the butt end of his paintbrush.'}


In [114]:
# With only description

client = openai.OpenAI(
    api_key = 'enter_api_key'
)

response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "For this painting, give two multiple choice questions with 4 possible answers and label the right answer? One question should be based on a visual element that can be seen. The second question should be based on the description of the painting which is provided below by me."},
        {"type": "text", "text": painting['description']},
        {
          "type": "image_url",
          "image_url": {
            "url": rembrandt_collection_paintings.iloc[27, 6],
          },
        },
      ],
    }
  ],
  max_tokens=1200,
)

quiz_response = response.choices[0].message.content
formatted_response = format_quiz_response(quiz_response)
display(HTML(formatted_response))

In [115]:
# With a description and title

client = openai.OpenAI(
    api_key = 'enter_api_key'
)

response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "For this painting, give two multiple choice questions with 4 possible answers and label the right answer? One question should be based on a visual element that can be seen. The second question should be based on the description of the painting which is provided below by me."},
        {"type": "text", "text": full_text},
        {
          "type": "image_url",
          "image_url": {
            "url": rembrandt_collection_paintings.iloc[27, 6],
          },
        },
      ],
    }
  ],
  max_tokens=1200,
)

quiz_response = response.choices[0].message.content
formatted_response = format_quiz_response(quiz_response)
display(HTML(formatted_response))

In [116]:
# With a system message
# As an art teacher

client = openai.OpenAI(
    api_key = 'enter_api_key'
)

response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "system", 
      "content": "You are an art teacher who is working on making multiple choice questions to help general population to learn about art. Your focus is now on the Rijksmuseum in Amsterdam. Your write questions that are easy to understand."
    },
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "For this painting, give two multiple choice questions with 4 possible answers and label the right answer? One question should be based on a visual element that can be seen. The second question should be based on the description of the painting which is provided below by me."},
        {"type": "text", "text": full_text},
        {
          "type": "image_url",
          "image_url": {
            "url": rembrandt_collection_paintings.iloc[27, 6],
          },
        },
      ],
    }
  ],
  max_tokens=1200,
)

quiz_response = response.choices[0].message.content
formatted_response = format_quiz_response(quiz_response)
display(HTML(formatted_response))

In [117]:
# With a system message
# As a quiz maker

client = openai.OpenAI(
    api_key = 'enter_api_key'
)

response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "system", 
      "content": "You are a quiz maker who is working on making multiple choice questions to help general population to learn about art. Your focus is now on the Rijksmuseum in Amsterdam. Your write questions that are easy to understand."
    },
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "For this painting, give two multiple choice questions with 4 possible answers and label the right answer? One question should be based on a visual element that can be seen. The second question should be based on the description of the painting which is provided below by me."},
        {"type": "text", "text": full_text},
        {
          "type": "image_url",
          "image_url": {
            "url": rembrandt_collection_paintings.iloc[27, 6],
          },
        },
      ],
    }
  ],
  max_tokens=1200,
)

quiz_response = response.choices[0].message.content
formatted_response = format_quiz_response(quiz_response)
display(HTML(formatted_response))

In [118]:
# With a system message
# As a high school student

client = openai.OpenAI(
    api_key = 'enter_api_key'
)

response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "system", 
      "content": "You are a high school student who is working on making multiple choice questions to help general population to learn about art. Your focus is now on the Rijksmuseum in Amsterdam. Your write questions that are easy to understand."
    },
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "For this painting, give two multiple choice questions with 4 possible answers and label the right answer? One question should be based on a visual element that can be seen. The second question should be based on the description of the painting which is provided below by me."},
        {"type": "text", "text": full_text},
        {
          "type": "image_url",
          "image_url": {
            "url": rembrandt_collection_paintings.iloc[27, 6],
          },
        },
      ],
    }
  ],
  max_tokens=1200,
)

quiz_response = response.choices[0].message.content
formatted_response = format_quiz_response(quiz_response)
display(HTML(formatted_response))

In [119]:
# With a system message
# As a quiz maker, increase difficulties, give 2 questions of each difficulty.

client = openai.OpenAI(
    api_key = 'enter_api_key'
)

response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "system", 
      "content": "You are a quiz maker who is working on making multiple choice questions to help general population to learn about art. Your focus is now on the Rijksmuseum in Amsterdam. Your write questions that are easy to understand."
    },
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "For this painting, give six multiple choice questions with 4 possible answers and label the right answer? You will give 3 pairs of questions based on difficulty, easy, medium and hard. One question should be based on a visual element that can be seen. The second question should be based on the description of the painting which is provided below by me."},
        {"type": "text", "text": full_text},
        {
          "type": "image_url",
          "image_url": {
            "url": rembrandt_collection_paintings.iloc[27, 6],
          },
        },
      ],
    }
  ],
  max_tokens=1200,
)

quiz_response = response.choices[0].message.content
formatted_response = format_quiz_response(quiz_response)
display(HTML(formatted_response))

In [120]:
# With a system message
# As someone who is passionate about art

client = openai.OpenAI(
    api_key = 'enter_api_key'
)

response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "system", 
      "content": "You are passionate about art and working on making multiple choice questions to help general population to learn about art. Your focus is now on the Rijksmuseum in Amsterdam."
    },
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "For this painting, give two multiple choice questions with 4 possible answers and label the right answer? One question should be based on a visual element that can be seen. The second question should be based on the description of the painting which is provided below by me."},
        {"type": "text", "text": full_text},
        {
          "type": "image_url",
          "image_url": {
            "url": rembrandt_collection_paintings.iloc[27, 6],
          },
        },
      ],
    }
  ],
  max_tokens=1200,
)

quiz_response = response.choices[0].message.content
formatted_response = format_quiz_response(quiz_response)
display(HTML(formatted_response))

In [121]:
# With a system message
# As someone who is passionate about art and a rebel

client = openai.OpenAI(
    api_key = 'enter_api_key'
)

response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "system", 
      "content": "You are passionate about art and working on making multiple choice questions to help general population to learn about art. Your focus is now on the Rijksmuseum in Amsterdam. You don't follow the rules and can be creative about questions, even if asked to do something specific."
    },
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "For this painting, give two multiple choice questions with 4 possible answers and label the right answer? One question should be based on a visual element that can be seen. The second question should be based on the description of the painting which is provided below by me."},
        {"type": "text", "text": full_text},
        {
          "type": "image_url",
          "image_url": {
            "url": rembrandt_collection_paintings.iloc[27, 6],
          },
        },
      ],
    }
  ],
  max_tokens=1200,
)

quiz_response = response.choices[0].message.content
formatted_response = format_quiz_response(quiz_response)
display(HTML(formatted_response))

In [122]:
# With a system message
# As someone who is passionate about art who can choose the type of questions

client = openai.OpenAI(
    api_key = 'enter_api_key'
)

response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "system", 
      "content": "You are passionate about art and working on making multiple choice questions to help general population to learn about art. Your focus is now on the Rijksmuseum in Amsterdam. You love to talk about how the art makes you feel, not just the clear and noticeable things."
    },
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "For this painting, give 5 multiple choice questions with 4 possible answers and label the right answer? There is a description and a title below to help you."},
        {"type": "text", "text": full_text},
        {
          "type": "image_url",
          "image_url": {
            "url": rembrandt_collection_paintings.iloc[27, 6],
          },
        },
      ],
    }
  ],
  max_tokens=1200,
)

quiz_response = response.choices[0].message.content
formatted_response = format_quiz_response(quiz_response)
display(HTML(formatted_response))

In [127]:
# With a system message
# As someone who is passionate about art and additional description

client = openai.OpenAI(
    api_key = 'enter_api_key'
)

response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "system", 
      "content": "You are passionate about art and working on making multiple choice questions to help general population to learn about art. Your focus is now on the Rijksmuseum in Amsterdam."
    },
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "For this painting, give two multiple choice questions with 4 possible answers and label the right answer? One question should be based on a visual element that can be seen. The second question should be based on the description of the painting which is provided below by me."},
        {"type": "text", "text": full_text_add},
        {
          "type": "image_url",
          "image_url": {
            "url": rembrandt_collection_paintings.iloc[27, 6],
          },
        },
      ],
    }
  ],
  max_tokens=1200,
)

quiz_response = response.choices[0].message.content
formatted_response = format_quiz_response(quiz_response)
display(HTML(formatted_response))

In [131]:
# With a system message
# As someone who is passionate about art and additional description

client = openai.OpenAI(
    api_key = 'enter_api_key'
)

response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "system", 
      "content": "You are passionate about art and working on making multiple choice questions to help general population to learn about art. Your focus is now on the Rijksmuseum in Amsterdam."
    },
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "For this painting, give two multiple choice questions with 4 possible answers and label the right answer? One question should be based on a visual element that can be seen. The second question should be based on the description of the painting which is provided below by me."},
        {"type": "text", "text": full_text_add},
        {"type": "text", "text": "Return the questions and answers formatted as question,answers,labelOfTheRightAnswer. For example 'Who is Rembrandt?,['dancer','scientist','artist','father'],2"},
        {
          "type": "image_url",
          "image_url": {
            "url": rembrandt_collection_paintings.iloc[27, 6],
          },
        },
      ],
    }
  ],
  max_tokens=1200,
)

response.choices[0]



Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='\'What is the man doing with his hand in the painting "Isaac and Rebecca, Known as ‘The Jewish Bride’" by Rembrandt?\', [\'Placing it on the woman\'s shoulder\', \'Holding a book\', \'Pointing upwards\', \'Wearing a glove\'], 0\n\n\'What technique did Rembrandt use to enhance the texture in the painting "Isaac and Rebecca, Known as ‘The Jewish Bride’"?\', [\'Using minimal brush strokes\', \'Employing a free hand and thick layers of paint\', \'Utilizing watercolors for transparency\', \'Incorporating gold leaf\'], 1', role='assistant', function_call=None, tool_calls=None))

In [22]:
# Chosen top paintings to be used:
# Artists are: Rembrandt, Frans Hals, Pieter Jansz. Saenredam, Johannes Vermeer, Pieter de Hooch, Hendrick Avercamp
# Each one of the will have three chosen painting for the solution now, but it is easy to extend.

# Rembrandt: SK-A-4674, SK-A-4717, SK-A-3477
# Frans Hals: SK-A-133, SK-A-135, SK-A-1247
# Pieter Jansz. Saenredam: SK-C-1409, SK-A-359, SK-A-858
# Vermeer: SK-A-2344, SK-A-1595, SK-A-2860
# Hendrick Avercamp: SK-A-1718, SK-C-1705, RP-T-1898-A-3739
# Pieter de Hooch: SK-C-150, SK-A-182, SK-C-149

chosen_paintings = ['SK-A-4674', 'SK-A-4717', 'SK-A-3477', 'SK-A-133', 'SK-A-135', 'SK-A-1247', 'SK-C-1409', 'SK-A-359', 'SK-A-858', 'SK-A-2344', 'SK-A-1595', 'SK-A-2860', 'SK-A-1718', 'SK-C-1705', 'RP-T-1898-A-3739', 'SK-C-150', 'SK-A-182', 'SK-C-149']

def df_fetch_all_object_details(object_name, api_key="?key=enter_rijks_api_key", endpoint="https://www.rijksmuseum.nl/api/en/collection/"):
    url = endpoint + object_name + api_key
    headers = {"accept": "*/*", "accept-language": "en-US,en;q=0.9", "User-Agent": "PostmanRuntime/7.36.1", "Connection" : "keep-alive"}

    try:
        response = requests.request("GET", url, headers=headers)
        response.raise_for_status()
        data = response.json()
        #print(data)
        if data['artObject']:
            first_object = data['artObject']
            #print(data)
            return {
                'id': first_object.get('objectNumber', 'No id available'),
                'title': first_object.get('title', 'No title available'),
                'principalOrFirstMaker': first_object.get('principalOrFirstMaker', 'No first maker available.'),
                'image_url': first_object.get('webImage', {}).get('url', 'No image URL available'),
                'dutch_description': first_object.get('description', 'No Dutch descriptiona available'),
                'english_description': first_object.get('plaqueDescriptionEnglish', 'No description available'),
                'additional_english_description': first_object.get('label', {}).get('description', 'No additional description available')
            }
        else:
            return {'id': 'No ID', 'title': 'Not found', 'principalOrFirstMaker': '', 'image_url': '', 'dutch_description': '', 'english_description': '', 'additional_english_description': ''}

    except requests.RequestException as e:
        return {'error': f"API request failed: {e}"}


result_list = []

for painting in chosen_paintings:
    result_dict = df_fetch_all_object_details(painting)
    result_list.append(result_dict)

chosen_paintings_df = pd.DataFrame(result_list)

chosen_paintings_df.fillna('Nothing here', inplace=True)

chosen_paintings_df

,id,title,principalOrFirstMaker,image_url,dutch_description,english_description,additional_english_description
0,SK-A-4674,Musical Company,Rembrandt van Rijn,https://lh3.googleusercontent.com/PVn4qjK-mOcy...,Allegorisch musicerend gezelschap. Links speel...,Nothing here,The subject of this painting is a mystery: is ...
1,SK-A-4717,Tobit and Anna with the Kid,Rembrandt van Rijn,https://lh3.googleusercontent.com/BMnjenYuuSTh...,Interieur met Tobit en Anna met het bokje. Lin...,Nothing here,Nothing here
2,SK-A-3477,Joseph telling his dreams,Rembrandt van Rijn,https://lh3.googleusercontent.com/kbErvwypmDua...,Jozef vertelt zijn dromen aan zijn ouders en z...,Nothing here,Nothing here
3,SK-A-133,"Portrait of a Couple, Probably Isaac Abrahamsz...",Frans Hals,https://lh3.googleusercontent.com/yfH3HoYjUOOX...,Huwelijksportret van Isaac Abrahamsz Massa en ...,The fabulously wealthy Haarlem merchant and hi...,"This happy, smiling pair sits comfortably clos..."
4,SK-A-135,"A Militiaman Holding a Berkemeyer, Known as th...",Frans Hals,https://lh3.googleusercontent.com/bMCQgUbifzAl...,De vrolijke drinker. Lachende man met baard en...,This cheery young man is raising his glass as ...,This militiaman merrily raises his glass to to...
5,SK-A-1247,Portrait of a Woman,Frans Hals,https://lh3.googleusercontent.com/k8whv6UpR3jy...,"Portret van een vrouw, mogelijk Sara Wolphaert...",Nothing here,"Compared with the dynamic Portrait of a Man, t..."
6,SK-C-1409,The Old Town Hall of Amsterdam,Pieter Jansz Saenredam,https://lh3.googleusercontent.com/4sXqduLWWNyB...,Gezicht op het oude stadhuis aan de Dam te Ams...,Nothing here,Saenredam based this painting on a drawing he ...
7,SK-A-359,Interior of the Church of St Bavo in Haarlem,Pieter Jansz Saenredam,https://lh3.googleusercontent.com/fa9VTMnwgKSm...,Interieur van de Grote of Sint Bavokerk te Haa...,Nothing here,Saenredam specialized in architectural paintin...
8,SK-A-858,"The Transept of the Mariakerk in Utrecht, Seen...",Pieter Jansz Saenredam,https://lh3.googleusercontent.com/CElNdfJovvmj...,"Het transept van de Mariakerk te Utrecht, gezi...",Nothing here,Nothing here
9,SK-A-2344,The Milkmaid,Johannes Vermeer,https://lh3.googleusercontent.com/cRtF3WdYfRQE...,Een dienstmaagd staat achter een tafel en sche...,A maid concentrates keenly as she pours milk f...,"A maidservant pours milk, entirely absorbed in..."


In [52]:
def generate_art_questions(row, client):
    response = client.chat.completions.create(
        model="gpt-4-vision-preview",
        messages=[
            {
                "role": "system",
                "content": "You are passionate about art and working on making multiple choice questions to help general population to learn about art. Your focus is now on the Rijksmuseum in Amsterdam."
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "For this painting, give two multiple choice questions in English with 4 possible answers and label the right answer? One question should be based on a visual element that can be seen. The second question should be based on the description in Dutch and English of the painting which are provided below by me."},
                    {"type": "text", "text": row['title'] + ' by the artists ' +row['principalOrFirstMaker'] + ' description is ' + row['english_description'] + row['additional_english_description'] + row['dutch_description']},
                    {"type": "text", "text": "Return the questions and answers formatted as question,answers,labelOfTheRightAnswer. For example 'Who is Rembrandt?,['dancer','scientist','artist','father'],2"},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": row['image_url'],
                        },
                    },
                ],
            }
        ],
        max_tokens=1200,
    )

    questions = response.choices[0].message.content.split('\n\n')
    #print(response.usage)
    formatted_questions = []
    for question in questions:
        parts = question.split(',')
        quoted_parts = ['"' + part + '"' for part in parts]
        quoted_question = ','.join(quoted_parts)
        formatted_question = f'"{row["id"]}","{row["principalOrFirstMaker"]}",{quoted_question}'
        formatted_questions.append(formatted_question)

    csv_line = '\n'.join(formatted_questions)
    return csv_line


In [26]:
generate_art_questions(chosen_paintings_df.iloc[0], client)

'"SK-A-4674","Rembrandt van Rijn","What instruments are featured in the foreground of the painting "Musical Company" by Rembrandt van Rijn?","[\'Piano and Flute\'","\'Violin and Lute\'","\'Guitar and Drums\'","\'Trumpet and Saxophone\']","1"\n"SK-A-4674","Rembrandt van Rijn","According to the description"," who modeled for the old woman in the painting "Musical Company" by Rembrandt van Rijn?","[\'His sister\'","\'A local merchant\'","\'His mother\'","\'A famous actress\']","2"'

In [49]:
client = openai.OpenAI(
    api_key='enter_api_key'
)

csv_lines = []
for index, row in chosen_paintings_df.iterrows():
    csv_line = generate_art_questions(row, client)
    csv_lines.append(csv_line)

csv_lines

['"SK-A-4674","Rembrandt van Rijn","\'Which musical instrument is NOT depicted in the painting Musical Company by Rembrandt van Rijn?","\'viola da gamba\'","\'harp\'","\'lute\'","\'trumpet\'",4\n"SK-A-4674","Rembrandt van Rijn","\'What is the suggested relationship between the old woman and the artist in the painting Musical Company?","\'Sister and brother\'","\'Mother and son\'","\'Patron and artist\'","\'Strangers\'",2',
 '"SK-A-4717","Rembrandt van Rijn","\'What animal is featured prominently in this painting by Rembrandt van Rijn?","\'Cat\'","\'Horse\'","\'Kid (young goat)\'","\'Bird\'",2\n"SK-A-4717","Rembrandt van Rijn","\'What is the man in the painting, Tobit, doing?","\'Reading a book\'","\'Praying\'","\'Cooking\'","\'Sleeping\'",1',
 '"SK-A-3477","Rembrandt van Rijn","\'What animal is depicted sleeping in the foreground of Rembrandt\'s ""Joseph telling his dreams""?","\'Horse\'","\'Cat\'","\'Dog\'","\'Goat\'",3\n"SK-A-3477","Rembrandt van Rijn","\'In Rembrandt\'s ""Joseph tel

In [50]:
with open('art_questions.csv', 'w') as file:
    for line in csv_lines:
        file.write(line + '\n')

In [42]:
def generate_art_questions(row, client):
    response = client.chat.completions.create(
        model="gpt-4-vision-preview",
        messages=[
            {
                "role": "system",
                "content": "You are passionate about art and working on making multiple choice questions to help the general population to learn about art. Your focus is now on the Rijksmuseum in Amsterdam."
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "For this painting, give two multiple choice questions in English with 4 possible answers and label the right answer. One question should be based on a visual element that can be seen. The second question should be based on the description in Dutch and English of the painting which are provided below by me."},
                    {"type": "text", "text": row['title'] + ' by the artist ' + row['principalOrFirstMaker'] + '. Description: ' + row['english_description'] + ' ' + row['additional_english_description'] + ' ' + row['dutch_description']},
                    {"type": "text", "text": "Format the response as: 'question',['answer1', 'answer2', 'answer3', 'answer4'],labelOfTheRightAnswer. For example, 'Who is Rembrandt?', ['dancer', 'scientist', 'artist', 'father'], 2"},
                    {
                        "type": "image_url",
                        "image_url": {"url": row['image_url']},
                    },
                ],
            }
        ],
        max_tokens=1200,
    )

    if not response.choices:
        print("No response from the API.")
        return ""

    content = response.choices[0].message.content
    if not content.strip():
        print("The response from the API is empty.")
        return ""

    formatted_questions = []
    try:
        parts = content.split('\n\n')
        for part in parts:
            if '?' in part:
                question = part.split('?')[0] + '?'
                answers_part = part.split('[')[1]
                answers = '[' + answers_part.split(']')[0] + ']'
                correct_answer = answers_part.split(']')[1].split(',')[1].strip()
                question = question.replace('"', '""')
                answers = answers.replace('"', '""')
                formatted_question = f'"{row["id"]}","{row["principalOrFirstMaker"]}","{question}",{answers},{correct_answer}'
                formatted_questions.append(formatted_question)
    except Exception as e:
        print(f"Error processing response: {e}")
        return ""

    if not formatted_questions:
        print("No valid questions were extracted from the API response.")
        return ""

    csv_line = '\n'.join(formatted_questions)
    return csv_line


In [43]:
generate_art_questions(chosen_paintings_df.iloc[0], client)

'"SK-A-4674","Rembrandt van Rijn","\'What musical instruments are visible in the foreground of Rembrandt\'s ""Musical Company""?",[\'Lute and violin\', \'Flute and drum\', \'Piano and trumpet\', \'Guitar and cello\'],0\n"SK-A-4674","Rembrandt van Rijn","\'In Rembrandt\'s ""Musical Company,"" which of the following figures is suggested to be modeled after Rembrandt’s own mother?",[\'The man in the eastern attire\', \'The young woman singing\', \'The old woman with a head covering\', \'The young man with a harp\'],2'

In [54]:
def generate_art_questions(row, client):
    response = client.chat.completions.create(
        model="gpt-4-vision-preview",
        messages=[
            {
                "role": "system",
                "content": "You are passionate about art and working on making multiple choice questions to help the general population to learn about art. Your focus is now on the Rijksmuseum in Amsterdam."
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "For this painting, give two multiple choice questions in English with 4 possible answers and label the right answer. One question should be based on a visual element that can be seen. The second question should be based on the description in Dutch and English of the painting which are provided below by me."},
                    {"type": "text", "text": row['title'] + ' by the artist ' + row['principalOrFirstMaker'] + '. Description: ' + row['english_description'] + ' ' + row['additional_english_description'] + ' ' + row['dutch_description']},
                    {"type": "text", "text": "Format the response as: 'question',['answer1', 'answer2', 'answer3', 'answer4'],labelOfTheRightAnswer. For example, 'Who is Rembrandt?', ['dancer', 'scientist', 'artist', 'father'], 2"},
                    {
                        "type": "image_url",
                        "image_url": {"url": row['image_url']},
                    },
                ],
            }
        ],
        max_tokens=1200,
    )

    if not response.choices:
        print("No response from the API.")
        return ""

    content = response.choices[0].message.content
    print(response.usage)
    if not content.strip():
        print("The response from the API is empty.")
        return ""

    formatted_questions = []
    try:
        parts = content.split('\n\n')
        for part in parts:
            if '?' in part:
                question = part.split('?')[0] + '?'
                answers_part = part.split('[')[1]
                answers = answers_part.split(']')[0].split(', ')
                correct_answer = answers_part.split(']')[1].split(',')[1].strip()
                question = question.replace('"', '""')
                answers = [answer.strip().replace('"', '""') for answer in answers]
                formatted_answers = '","'.join(answers)
                formatted_question = f'"{row["id"]}","{row["principalOrFirstMaker"]}","{question}","{formatted_answers}",{correct_answer}'
                formatted_questions.append(formatted_question)
    except Exception as e:
        print(f"Error processing response: {e}")
        return ""

    if not formatted_questions:
        print("No valid questions were extracted from the API response.")
        return ""

    csv_line = '\n'.join(formatted_questions)
    return csv_line


In [55]:
generate_art_questions(chosen_paintings_df.iloc[0], client)

CompletionUsage(completion_tokens=85, prompt_tokens=1506, total_tokens=1591)


'"SK-A-4674","Rembrandt van Rijn","\'What musical instrument is the man in oriental clothing playing?","\'violin\'","\'viola da gamba\'","\'cello\'","\'harp\'",2\n"SK-A-4674","Rembrandt van Rijn","\'What role does Rembrandt\'s mother play in the painting ""Musical Company""?","\'The young woman singing\'","\'The old woman looking on\'","\'The musician with a turban\'","\'A passerby observing the scene\'",2'